In [276]:
from algs_lib import *


In [278]:
def clip_to_threshold(vec, c):
    curr_norm = np.linalg.norm(vec)
#     print(curr_norm, c)
    if curr_norm <= c:
        return vec
#     print('reached')
    clip_ratio = c / curr_norm
    return [vec[i]*clip_ratio for i in range(len(vec))]

In [280]:
def add_noise(scale):
    return np.random.laplace(0, scale)
# global sensitivity is C/n i think?
# so scale should be (C/n) / \epsilon per elem?

In [282]:
def calc_posterior(mi, prior=0.5, prec = 100000):
    test_vals = [x / prec for x in range(1, prec)]
    max_t = None
    for t in test_vals:
        if t*np.log(t/prior)+(1-t)*np.log((1-t)/(1-prior)) <= mi:
            if  max_t is None or t > max_t:
                max_t = t
    return max_t

def dp_epsilon_to_posterior_success(epsilon):
    return 1 - 1./(1+np.exp(epsilon))

def dp_ps_to_epsilon(ps):
    return np.log(ps / (1-ps))

In [284]:
calc_posterior(1./64, prior=0.01)




0.03213

In [286]:
train_x, train_y, test_x, test_y, num_classes, train_len = gen_iris(normalize=True)
subsample_rate = int(0.5*train_len)


In [288]:

est_y = {}
est_x = {}
prev_ests = None
seed = 743895091 # randomly generated seed for reproducibility
num_trials = 1024
curr_trial = 0
max_noises = {}

while curr_trial < num_trials:
    shuffled_inds = shuffle(list(range(len(train_x))))
    # shuffled_x, shuffled_y = shuffle(train_x, train_y)
    x, y = train_x[shuffled_inds], train_y[shuffled_inds]
    x1, y1 = x[:subsample_rate], y[:subsample_rate]
    x2, y2 = x[subsample_rate:], y[subsample_rate:]
    inds_1, inds_2 = shuffled_inds[:subsample_rate], shuffled_inds[subsample_rate:]
    est_x[curr_trial] = inds_1
    est_x[curr_trial + 1] = inds_2
    curr_trial += 2
    

In [290]:
len(est_x)

1024

In [291]:
test_ind = 0

xs_in = [k for k in est_x if ind in est_x[k]]
xs_out = [k for k in est_x if ind not in est_x[k]]

for set_ind in xs_in:
    s_x = set(est_x[set_ind])

    best_match, best_overlap = None, None
    for k in xs_out:
        indices = est_x[k]
        overlap = len(set(indices).intersection(s_x))
        if best_overlap is None or overlap > best_overlap:
            best_overlap = overlap
            best_match = k

In [292]:
len(xs_in), len(xs_out)

(512, 512)

In [293]:
xs_in

[1,
 3,
 4,
 7,
 8,
 10,
 13,
 14,
 17,
 18,
 21,
 23,
 24,
 27,
 29,
 30,
 32,
 35,
 36,
 38,
 40,
 42,
 45,
 47,
 49,
 50,
 52,
 55,
 57,
 59,
 61,
 62,
 65,
 67,
 68,
 70,
 72,
 74,
 76,
 79,
 81,
 82,
 84,
 87,
 89,
 91,
 93,
 94,
 97,
 99,
 100,
 102,
 105,
 106,
 108,
 110,
 113,
 115,
 116,
 119,
 120,
 123,
 124,
 127,
 128,
 130,
 133,
 134,
 136,
 139,
 140,
 143,
 144,
 147,
 148,
 151,
 152,
 154,
 156,
 159,
 161,
 163,
 165,
 167,
 168,
 170,
 173,
 174,
 177,
 178,
 180,
 182,
 184,
 187,
 188,
 190,
 193,
 195,
 197,
 198,
 200,
 203,
 205,
 207,
 209,
 210,
 213,
 215,
 216,
 219,
 221,
 223,
 225,
 226,
 229,
 231,
 232,
 234,
 236,
 239,
 240,
 243,
 245,
 246,
 248,
 250,
 253,
 254,
 256,
 259,
 260,
 262,
 264,
 267,
 268,
 271,
 273,
 274,
 276,
 278,
 280,
 283,
 285,
 286,
 288,
 291,
 292,
 295,
 297,
 299,
 300,
 303,
 305,
 307,
 308,
 311,
 312,
 315,
 317,
 319,
 320,
 323,
 324,
 326,
 328,
 331,
 332,
 335,
 336,
 339,
 340,
 342,
 345,
 346,
 349,
 351,

In [294]:
len(est_x)

1024

In [295]:
len(set(est_x[847]).intersection(s_x))

645

In [296]:
mi_range = [0.25, 1/16., 0.015625]
posterior_success_rates = [calc_posterior(mi) for mi in mi_range]
epsilon_vals = [dp_ps_to_epsilon(ps) for ps in posterior_success_rates]

print(epsilon_vals)
print([x for x in posterior_success_rates])

[1.6426117097961406, 0.7304317044395013, 0.3563228120191924]
[0.83789, 0.6749, 0.58815]


In [297]:
# DP MEAN
train_x, train_y, test_x, test_y, num_classes, train_len = gen_iris(normalize=True)
true_mean = np.average(train_x, axis=0)

dp_dists = {}
num_trials = 1000
for eps in epsilon_vals:
    avg_dist_dp = {}
    for i in range(1, 17):
        clip_budget = i / 10
        clipped_train_x = [clip_to_threshold(train_x[i], clip_budget) for i in range(len(train_x))]
        released_mean = np.average(clipped_train_x, axis=0)
        clip_dist = np.linalg.norm(released_mean - true_mean)
        dist = 0.
        sensitivity = clip_budget / train_len
        for _ in range(num_trials):
            released_mean = np.average(clipped_train_x, axis=0)
            for ind in range(len(released_mean)):
                sensitivity = clip_budget / train_len
                released_mean[ind] += add_noise(sensitivity/eps)
            dist += np.linalg.norm(released_mean - true_mean)
        dist /= num_trials
        avg_dist_dp[i] = (clip_dist, dist)
    dp_key = min(avg_dist_dp.items(), key=lambda x: x[1][1])[0]
    print(dp_key)
    dp_dists[eps] = avg_dist_dp[dp_key]


15
14
13


In [298]:
dp_dists

{1.6426117097961406: (0.003760138063449314, 0.02258502174391287),
 0.7304317044395013: (0.01164606776365974, 0.04969819520789844),
 0.3563228120191924: (0.02706865690970566, 0.09332670532636303)}

In [299]:
from scipy import stats

In [300]:
with open(f'test/iris_mean_noise_ind=True.pkl', 'rb') as f:
    noise, xs = pickle.load(f)[0.5] # keyed by MI, default is 0.5
print(sum(noise.values()))   
with open(f'data/iris_mean_noise_ind=True.pkl', 'rb') as f:
    noise, xs = pickle.load(f)[0.5] # keyed by MI, default is 0.5
print(sum(noise.values()))   

with open(f'test/iris_mean_noise_ind=False.pkl', 'rb') as f:
    noise, xs = pickle.load(f)[0.5] # keyed by MI, default is 0.5
print(sum(noise.values()))   
with open(f'data/iris_mean_noise_ind=False.pkl', 'rb') as f:
    noise, xs = pickle.load(f)[0.5] # keyed by MI, default is 0.5
print(sum(noise.values()))   

0.00032252681632139
0.0003155611516452299
0.011478844241318502
0.009559438148884583


In [301]:
# PAC MEAN LOCAL
train_x, train_y, test_x, test_y, num_classes, train_len = gen_iris(normalize=True)
true_mean = np.average(train_x, axis=0)
subsample_rate = int(0.5*train_len)

with open(f'test/iris_mean_noise_ind=True.pkl', 'rb') as f:
    noise, xs = pickle.load(f)[0.5] # keyed by MI, default is 0.5


pac_dists = {}
num_trials = 1000
for mi in mi_range:
    scaled_noise = {k: noise[k] * (0.5 / mi) for k in noise}
    iso_noise = max(scaled_noise.values())
    iso_scaled = {k: iso_noise for k in noise}
    avg_dist_pac = 0
    avg_iso_dist_pac = 0
    subsampled_dist = 0
    for _ in range(num_trials):
        choice = np.random.choice(range(1024))
        shuffled_x1, shuffled_y1 = train_x[xs[choice]], train_y[xs[choice]]
        released_mean = np.average(shuffled_x1, axis=0)
        subsampled_dist += np.linalg.norm(released_mean - true_mean)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=scaled_noise[ind])
            released_mean[ind] += c
        avg_dist_pac += np.linalg.norm(released_mean - true_mean)
    subsampled_dist /= num_trials
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=iso_scaled[ind])
            released_mean[ind] += c
        avg_iso_dist_pac += np.linalg.norm(released_mean - true_mean)
    
    avg_iso_dist_pac /= num_trials
    avg_dist_pac /= num_trials

#     subsampled_dist /= num_trials
    pac_dists[mi] = (subsampled_dist, avg_dist_pac, avg_iso_dist_pac)
print(pac_dists)

{0.25: (0.04478935155240638, 0.044786037380054995, 0.043328539082840026), 0.0625: (0.045302130329422, 0.04532076107567905, 0.04433696289766765), 0.015625: (0.04465053429919298, 0.04493462221946383, 0.04446955524605017)}


In [302]:
# PAC MEAN GLOBAL
train_x, train_y, test_x, test_y, num_classes, train_len = gen_iris(normalize=True)
true_mean = np.average(train_x, axis=0)
subsample_rate = int(0.5*train_len)

with open(f'test/iris_mean_noise_ind=False.pkl', 'rb') as f:
    noise, xs = pickle.load(f)[0.5] # keyed by MI, default is 0.5


pac_dists = {}
num_trials = 1000
for mi in mi_range:
    scaled_noise = {k: noise[k] * (0.5 / mi) for k in noise}
    iso_noise = max(scaled_noise.values())
    iso_scaled = {k: iso_noise for k in noise}
    avg_dist_pac = 0
    avg_iso_dist_pac = 0
    subsampled_dist = 0
    for _ in range(num_trials):
        choice = np.random.choice(range(1024))
        shuffled_x1, shuffled_y1 = train_x[xs[choice]], train_y[xs[choice]]
        released_mean = np.average(shuffled_x1, axis=0)
        subsampled_dist += np.linalg.norm(released_mean - true_mean)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=scaled_noise[ind])
            released_mean[ind] += c
        avg_dist_pac += np.linalg.norm(released_mean - true_mean)
    subsampled_dist /= num_trials
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=iso_scaled[ind])
            released_mean[ind] += c
        avg_iso_dist_pac += np.linalg.norm(released_mean - true_mean)
    
    avg_iso_dist_pac /= num_trials
    avg_dist_pac /= num_trials

#     subsampled_dist /= num_trials
    pac_dists[mi] = (subsampled_dist, avg_dist_pac, avg_iso_dist_pac)
print(pac_dists)

{0.25: (0.04653742410836648, 0.04821759917638942, 0.04638160286888963), 0.0625: (0.04768789839040323, 0.06744404898300899, 0.07158395125256205), 0.015625: (0.046432717720573835, 0.1806701384927753, 0.2172688948316337)}


In [303]:
0.191/0.161

1.186335403726708

In [304]:
train_x, train_y, test_x, test_y, num_classes, train_len = gen_bean(normalize=True)

In [305]:
true_mean = np.average(train_x, axis=0)

norms = [np.linalg.norm(x) for x in train_x]
print(max(norms))

3.0829053788327507


In [306]:
true_mean

array([0.1399897 , 0.22680687, 0.24606585, 0.23724242, 0.39587704,
       0.76689908, 0.13704124, 0.22560331, 0.62619689, 0.9002929 ,
       0.76622798, 0.46101523, 0.49242476, 0.37227223, 0.41488435,
       0.91073302])

In [307]:
# DP MEAN
dp_dists = {}
num_trials = 1000

for eps in epsilon_vals:
    avg_dist_dp = {}
    for i in range(1, 31):
        clip_budget = i/10
        clipped_train_x = [clip_to_threshold(train_x[i], clip_budget) for i in range(len(train_x))]
        released_mean = np.average(clipped_train_x, axis=0)
        clip_dist = np.linalg.norm(released_mean - true_mean)
        dist = 0.
        for _ in range(num_trials):
            released_mean = np.average(clipped_train_x, axis=0)
            for ind in range(len(released_mean)):
                sensitivity = clip_budget / train_len 
                released_mean[ind] += add_noise(sensitivity / eps)
            dist += np.linalg.norm(released_mean - true_mean)
        dist /= num_trials
        avg_dist_dp[i] = (clip_dist, dist)
    dp_key = min(avg_dist_dp.items(), key=lambda x: x[1][1])[0]
    dp_dists[eps] = avg_dist_dp[dp_key]


In [308]:
dp_dists

{1.6426117097961406: (7.189599906288743e-05, 0.0010187629845155631),
 0.7304317044395013: (0.00031403038825697635, 0.0022340125713306365),
 0.3563228120191924: (0.0009807389338486732, 0.004459543287289186)}

In [309]:
# PAC MEAN LOCAL
train_x, train_y, test_x, test_y, num_classes, train_len = gen_bean(normalize=True)
true_mean = np.average(train_x, axis=0)
subsample_rate = int(0.5*train_len)

with open(f'test/bean_mean_noise_ind=True.pkl', 'rb') as f:
    noise, xs = pickle.load(f)[0.5] # keyed by MI, default is 0.5


pac_dists = {}
num_trials = 1000
for mi in mi_range:
    scaled_noise = {k: noise[k] * (0.5 / mi) for k in noise}
    iso_noise = max(scaled_noise.values())
    iso_scaled = {k: iso_noise for k in noise}
    avg_dist_pac = 0
    avg_iso_dist_pac = 0
    subsampled_dist = 0
    for _ in range(num_trials):
        choice = np.random.choice(range(1024))
        shuffled_x1, shuffled_y1 = train_x[xs[choice]], train_y[xs[choice]]
        released_mean = np.average(shuffled_x1, axis=0)
        subsampled_dist += np.linalg.norm(released_mean - true_mean)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=scaled_noise[ind])
            released_mean[ind] += c
        avg_dist_pac += np.linalg.norm(released_mean - true_mean)
    subsampled_dist /= num_trials
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=iso_scaled[ind])
            released_mean[ind] += c
        avg_iso_dist_pac += np.linalg.norm(released_mean - true_mean)
    
    avg_iso_dist_pac /= num_trials
    avg_dist_pac /= num_trials

#     subsampled_dist /= num_trials
    pac_dists[mi] = (subsampled_dist, avg_dist_pac, avg_iso_dist_pac)
print(pac_dists)

{0.25: (0.0054849685943179766, 0.00548553984894541, 0.0053526367754119), 0.0625: (0.0054209992195542196, 0.005422277042560301, 0.005413025224079141), 0.015625: (0.005431260403192176, 0.005447635074952403, 0.00542849887481953)}


In [310]:
# PAC MEAN GLOBAL
train_x, train_y, test_x, test_y, num_classes, train_len = gen_bean(normalize=True)
true_mean = np.average(train_x, axis=0)
subsample_rate = int(0.5*train_len)

with open(f'test/bean_mean_noise_ind=False.pkl', 'rb') as f:
    noise, xs = pickle.load(f)[0.5] # keyed by MI, default is 0.5


pac_dists = {}
num_trials = 1000
for mi in mi_range:
    scaled_noise = {k: noise[k] * (0.5 / mi) for k in noise}
    iso_noise = max(scaled_noise.values())
    iso_scaled = {k: iso_noise for k in noise}
    avg_dist_pac = 0
    avg_iso_dist_pac = 0
    subsampled_dist = 0
    for _ in range(num_trials):
        choice = np.random.choice(range(1024))
        shuffled_x1, shuffled_y1 = train_x[xs[choice]], train_y[xs[choice]]
        released_mean = np.average(shuffled_x1, axis=0)
        subsampled_dist += np.linalg.norm(released_mean - true_mean)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=scaled_noise[ind])
            released_mean[ind] += c
        avg_dist_pac += np.linalg.norm(released_mean - true_mean)
    subsampled_dist /= num_trials
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=iso_scaled[ind])
            released_mean[ind] += c
        avg_iso_dist_pac += np.linalg.norm(released_mean - true_mean)
    
    avg_iso_dist_pac /= num_trials
    avg_dist_pac /= num_trials

#     subsampled_dist /= num_trials
    pac_dists[mi] = (subsampled_dist, avg_dist_pac, avg_iso_dist_pac)
print(pac_dists)

{0.25: (0.005273732173374416, 0.005284740797201349, 0.005366049471830891), 0.0625: (0.005301985824324389, 0.005420389190747994, 0.005549304107713246), 0.015625: (0.00534384657254746, 0.006991745299157017, 0.008055822107875118)}


In [311]:
pac_dists

{0.25: (0.005273732173374416, 0.005284740797201349, 0.005366049471830891),
 0.0625: (0.005301985824324389, 0.005420389190747994, 0.005549304107713246),
 0.015625: (0.00534384657254746, 0.006991745299157017, 0.008055822107875118)}

In [312]:
pac_dists

{0.25: (0.005273732173374416, 0.005284740797201349, 0.005366049471830891),
 0.0625: (0.005301985824324389, 0.005420389190747994, 0.005549304107713246),
 0.015625: (0.00534384657254746, 0.006991745299157017, 0.008055822107875118)}

In [313]:
# RICE

In [314]:
train_x, train_y, test_x, test_y, num_classes, train_len = gen_rice(normalize=True)

In [315]:
true_mean = np.average(train_x, axis=0)

norms = [np.linalg.norm(x) for x in train_x]
print(max(norms))

2.3296929410457268


In [316]:
true_mean

array([0.44901231, 0.50102237, 0.46286265, 0.55690775, 0.64194394,
       0.45831109, 0.44831546])

In [317]:
# DP MEAN
dp_dists = {}
num_trials = 1000

for eps in epsilon_vals:
    avg_dist_dp = {}
    for i in range(1, 24):
        clip_budget = i / 10
        clipped_train_x = [clip_to_threshold(train_x[i], clip_budget) for i in range(len(train_x))]
        released_mean = np.average(clipped_train_x, axis=0)
        clip_dist = np.linalg.norm(released_mean - true_mean)
        dist = 0.
        for _ in range(num_trials):
            released_mean = np.average(clipped_train_x, axis=0)
            for ind in range(len(released_mean)):
                sensitivity = clip_budget / train_len 
                released_mean[ind] += add_noise(sensitivity / eps)
            dist += np.linalg.norm(released_mean - true_mean)
        dist /= num_trials
        avg_dist_dp[i] = (clip_dist, dist)
    dp_key = min(avg_dist_dp.items(), key=lambda x: x[1][1])[0]
    dp_dists[eps] = avg_dist_dp[dp_key]


In [318]:
dp_dists

{1.6426117097961406: (0.0003743330012744167, 0.0017075091220344863),
 0.7304317044395013: (0.0003743330012744167, 0.0037607925542387103),
 0.3563228120191924: (0.0014293079056475226, 0.007657317501067411)}

In [319]:
# PAC MEAN LOCAL
train_x, train_y, test_x, test_y, num_classes, train_len = gen_rice(normalize=True)
true_mean = np.average(train_x, axis=0)
subsample_rate = int(0.5*train_len)

with open(f'test/rice_mean_noise_ind=True.pkl', 'rb') as f:
    noise, xs = pickle.load(f)[0.5] # keyed by MI, default is 0.5


pac_dists = {}
num_trials = 1000
for mi in mi_range:
    scaled_noise = {k: noise[k] * (0.5 / mi) for k in noise}
    iso_noise = max(scaled_noise.values())
    iso_scaled = {k: iso_noise for k in noise}
    avg_dist_pac = 0
    avg_iso_dist_pac = 0
    subsampled_dist = 0
    for _ in range(num_trials):
        choice = np.random.choice(range(1024))
        shuffled_x1, shuffled_y1 = train_x[xs[choice]], train_y[xs[choice]]
        released_mean = np.average(shuffled_x1, axis=0)
        subsampled_dist += np.linalg.norm(released_mean - true_mean)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=scaled_noise[ind])
            released_mean[ind] += c
        avg_dist_pac += np.linalg.norm(released_mean - true_mean)
    subsampled_dist /= num_trials
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=iso_scaled[ind])
            released_mean[ind] += c
        avg_iso_dist_pac += np.linalg.norm(released_mean - true_mean)
    
    avg_iso_dist_pac /= num_trials
    avg_dist_pac /= num_trials

#     subsampled_dist /= num_trials
    pac_dists[mi] = (subsampled_dist, avg_dist_pac, avg_iso_dist_pac)
print(pac_dists)

{0.25: (0.007757612820548558, 0.007757542060921992, 0.007956547348488422), 0.0625: (0.007610703757303351, 0.007610691069106876, 0.007484078527585295), 0.015625: (0.008080336078943519, 0.008088101758080578, 0.0078917097872143)}


In [320]:
# PAC MEAN GLOBAL
train_x, train_y, test_x, test_y, num_classes, train_len = gen_rice(normalize=True)
true_mean = np.average(train_x, axis=0)
subsample_rate = int(0.5*train_len)

with open(f'test/rice_mean_noise_ind=False.pkl', 'rb') as f:
    noise, xs = pickle.load(f)[0.5] # keyed by MI, default is 0.5


pac_dists = {}
num_trials = 1000
for mi in mi_range:
    scaled_noise = {k: noise[k] * (0.5 / mi) for k in noise}
    iso_noise = max(scaled_noise.values())
    iso_scaled = {k: iso_noise for k in noise}
    avg_dist_pac = 0
    avg_iso_dist_pac = 0
    subsampled_dist = 0
    for _ in range(num_trials):
        choice = np.random.choice(range(1024))
        shuffled_x1, shuffled_y1 = train_x[xs[choice]], train_y[xs[choice]]
        released_mean = np.average(shuffled_x1, axis=0)
        subsampled_dist += np.linalg.norm(released_mean - true_mean)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=scaled_noise[ind])
            released_mean[ind] += c
        avg_dist_pac += np.linalg.norm(released_mean - true_mean)
    subsampled_dist /= num_trials
    for _ in range(num_trials):
        shuffled_x1, shuffled_y1 = shuffle(train_x, train_y)
        shuffled_x1, shuffled_y1 = get_samples_safe(shuffled_x1, shuffled_y1, num_classes, subsample_rate)
        released_mean = np.average(shuffled_x1, axis=0)
        for ind in range(len(released_mean)):
            c = np.random.normal(0, scale=iso_scaled[ind])
            released_mean[ind] += c
        avg_iso_dist_pac += np.linalg.norm(released_mean - true_mean)
    
    avg_iso_dist_pac /= num_trials
    avg_dist_pac /= num_trials

#     subsampled_dist /= num_trials
    pac_dists[mi] = (subsampled_dist, avg_dist_pac, avg_iso_dist_pac)
print(pac_dists)

{0.25: (0.00748655011901608, 0.007504349828626526, 0.007806994039811815), 0.0625: (0.0075344489473279635, 0.007728619489802331, 0.0077494319984033125), 0.015625: (0.0074941816725075956, 0.009924254506241492, 0.011071368162478858)}


In [321]:
pac_dists

{0.25: (0.00748655011901608, 0.007504349828626526, 0.007806994039811815),
 0.0625: (0.0075344489473279635, 0.007728619489802331, 0.0077494319984033125),
 0.015625: (0.0074941816725075956, 0.009924254506241492, 0.011071368162478858)}

In [322]:
with open(f'test/iris_mean_noise_ind=True.pkl', 'rb') as f:
    noise, xs = pickle.load(f)[0.5] # keyed by MI, default is 0.5

In [232]:
print(sum(noise.values()))

0.00032252681632139


In [233]:
with open(f'mean_data/iris_mean_noise_ind=True.pkl', 'rb') as f:
    noise, xs = pickle.load(f)[0.5] # keyed by MI, default is 0.5
    
print(sum(noise.values()))

0.0003155611516452299
